In [8]:
using  Plots, Gen

In [6]:
SubChunkSize = 20
function DiffrenceIndex(i)
    return(div(i,SubChunkSize,RoundUp))
end
function SubXDif(xs,i)
    return(xs[i] - xs[div(i-1,SubChunkSize,RoundDown)*SubChunkSize+1])
end

SubXDif (generic function with 1 method)

In [9]:
#ysOfseted
function (trace::Trace; title="")
    trace = serialize_trace(trace)
    #Graph points
    outliers = [pt for (pt, outlier) in zip(trace[:points], trace[:outliers]) if outlier]
    inliers =  [pt for (pt, outlier) in zip(trace[:points], trace[:outliers]) if !outlier]
    PLT = Plots.scatter(map(first, inliers), map(last, inliers), markercolor="blue", label=nothing, title=title) 
    PLT = Plots.scatter!(map(first, outliers), map(last, outliers), markercolor="red", label=nothing)

    #Graph Line
    PLT = Plots.plot!(trace[:xs], trace[:ys], color = "black", lw = 3, label = nothing)
    return PLT
end

#28 (generic function with 1 method)


function serialize_trace(trace)
    (xs,) = Gen.get_args(trace)
    n = length(xs)
    NumChunks = div(n, SubChunkSize, RoundUp)
    Dict(:slope => [trace[(:slope, i)] for i in 1:NumChunks],
         #:SubSlope => [trace[(:SubSlope, i)] for i in 1:NumChunks],
         :inlier_std => [trace[(:noise, i)] for i in 1:NumChunks],
         :points => zip(xs, [trace[:data => i => :y] for i in 1:length(xs)]),
         :xs => xs,
         :Buffer_y => trace[:Buffer_y],
         :Buffer_SubSlope => trace[:Buffer_SubSlope],
         :outliers => [trace[:data => i => :is_outlier] for i in 1:length(xs)])
end

    #GraphLine
    n = length(trace[:xs])
    NumChunks = div(n, SubChunkSize, RoundUp)
    SubSlopes = cumsum(pushfirst!([SubChunkSize*trace[:slope][i] for i=1:NumChunks],trace[:Buffer_SubSlope]))
    XFunc(n)=(trace[:slope][DiffrenceIndex(n)]) * SubXDif(trace[:xs],n)^2 +
             (SubSlopes[DiffrenceIndex(n)]) * SubXDif(trace[:xs],n)

    
    yInter = cumsum(
        pushfirst!(
            [XFunc(i*SubChunkSize) - XFunc((i-1)*SubChunkSize+1) for i=1:(NumChunks-1)],
            trace[:Buffer_y])
            )
    
    LeftX = trace[:xs][1]
    LeftY = yInter[1]
    for i = 2:n
        RightX =  trace[:xs][i]
        RightY = XFunc(i) + yInter[DiffrenceIndex(i)]
        PLT = Plots.plot!([LeftX, RightX], [LeftY, RightY], color = "black", lw = 3, label = nothing)
        LeftX = RightX
        LeftY = RightY
    end